## Linking without deduplication

A simple record linkage model using the `link_only` [link type](https://moj-analytical-services.github.io/splink/settings_dict_guide.html#link_type).

In [1]:
from splink.datasets import splink_datasets
df = splink_datasets.fake_1000

# Split a simple dataset into two, separate datasets which can be linked together.
df_l = df.sample(frac=0.5)
df_r = df.drop(df_l.index)

df_l.head(2)

,unique_id,first_name,surname,dob,city,email,cluster
932,932,Luke,Robinssno,1971-11-18,Coventry,lrobinson@wolf.org,233
440,440,Campbell,Jacob,1987-06-06,London,j.c65@ortiz.com,113


In [2]:
from splink.duckdb.linker import DuckDBLinker
from splink.duckdb.blocking_rule_library import block_on
import splink.duckdb.comparison_library as cl
import splink.duckdb.comparison_template_library as ctl


settings = {
    "link_type": "link_only",
    "blocking_rules_to_generate_predictions": [
        block_on("first_name"),
        block_on("surname"),
    ],
    "comparisons": [
        ctl.name_comparison("first_name",),
        ctl.name_comparison("surname"),
        ctl.date_comparison("dob", cast_strings_to_date=True),
        cl.exact_match("city", term_frequency_adjustments=True),
        ctl.email_comparison("email", include_username_fuzzy_level=False),
    ],       
}

In [3]:
linker = DuckDBLinker([df_l, df_r], settings, input_table_aliases=["df_left", "df_right"])
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email"
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)


Probability two random records match is estimated to be  0.00332.
This means that amongst all possible pairwise record comparisons, one in 301.20 are expected to match.  With 250,000 total possible comparisons, we expect a total of around 830.00 matching pairs


In [4]:
linker.estimate_u_using_random_sampling(max_pairs=1e6, seed=1)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [5]:
session_dob = linker.estimate_parameters_using_expectation_maximisation(block_on("dob"))
session_email = linker.estimate_parameters_using_expectation_maximisation(block_on("email"))
session_first_name = linker.estimate_parameters_using_expectation_maximisation(block_on("first_name"))


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."dob" = r."dob"

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob

Iteration 1: Largest change in params was -0.393 in the m_probability of surname, level `Exact match surname`
Iteration 2: Largest change in params was 0.11 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0321 in probability_two_random_records_match
Iteration 4: Largest change in params was 0.0114 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.00471 in probability_two_random_records_match
Iteration 6: Largest change in params was 0.00209 in probability_two_random_records_match
Iteration 7: Largest change in params was 0.000961 in probability_two_random_rec

In [6]:
results = linker.predict(threshold_match_probability=0.9)

In [7]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,surname_l,...,dob_l,dob_r,gamma_dob,city_l,city_r,gamma_city,email_l,email_r,gamma_email,match_key
0,18.627558,0.999998,df_left,df_right,932,933,Luke,Luke,4,Robinssno,...,1971-11-18,1971-11-18,5,Coventry,Coventry,1,lrobinson@wolf.org,lrobinson@wolf.org,3,0
1,10.844350,0.999456,df_left,df_right,799,802,Arlo,Arlo,4,NaN,...,2007-02-17,2007-03-19,3,Southend-on-Sea,NaN,-1,awt.thompson@com,aw@thompson.com,1,0
2,10.066566,0.999068,df_left,df_right,68,70,Reggie,Reggie,4,NaN,...,1996-08-05,1986-09-09,1,Coventry,Coventry,1,r.m51@leonard.net,r.m51@leonard.net,3,0
3,13.497639,0.999914,df_left,df_right,485,486,Hannah,Hannah,4,Daivs,...,1992-07-24,1993-07-24,4,Birmingham,Birmingham,1,hannahd14@cardenas.com,NaN,-1,0
4,11.077715,0.999538,df_left,df_right,28,29,Thomas,Thomas,4,Gabriel,...,1976-09-15,1976-08-15,4,London,NaN,-1,gabriel.t54@nichols.info,NaN,-1,0
